In [ ]:
import pandas as pd
pd.set_option('display.max_rows', 100)

def add_track_features(tracks, fps=59.94, snap_frame=10):
    """
    Add column features helpful for syncing with video data.
    """
    tracks = tracks.copy()
    tracks["game_play"] = tracks["gameKey"].astype("str") + "_" + tracks["playID"].astype("str").str.zfill(6)
    tracks["time"] = pd.to_datetime(tracks["time"])
    snap_dict = tracks.query('event == "ball_snap"').groupby("game_play")["time"].first().to_dict()
    tracks["snap"] = tracks["game_play"].map(snap_dict)
    tracks["isSnap"] = tracks["snap"] == tracks["time"]
    tracks["team"] = tracks["player"].str[0].replace("H", "Home").replace("V", "Away")
    tracks["snap_offset"] = (tracks["time"] - tracks["snap"]).astype("timedelta64[ms]") / 1_000
    # Estimated video frame
    tracks["est_frame"] = ((tracks["snap_offset"] * fps) + snap_frame).round().astype("int")
    
    cols = ['x', 'y', 's', 'a', 'dis', 'o', 'dir']
    return tracks

tracks = pd.read_csv('../input/nfl-health-and-safety-helmet-assignment/train_player_tracking.csv')
tracks = add_track_features(tracks)

In [ ]:
counts = tracks.groupby(['game_play', 'est_frame'])['player'].agg(len)
counts[counts < 22].reset_index().sort_values('player')